In [281]:
import numpy as np
import tensorflow as tf
import gym
from collections import deque

In [282]:
# 하이퍼 파라미터
gridSize = 7
nbActions = gridSize * gridSize
nbStates = gridSize * gridSize

LEARNING_RATE = 0.005
INPUT = nbStates
OUTPUT = nbActions
DISCOUNT = 0.99

winReward = 1
hadicapReward = -10

STONE_NONE = 0
STONE_PLAYER1 = 1
STONE_PLAYER2 = -1
STONE_MAX = 5

In [283]:
hiddenSize = 100
maxMemory = 500
batchSize = 50
epoch = 100
epsilonStart = 1
epsilonDiscount = 0.999
epsilonMinimumValue = 0.1
discount = 0.9
learningRate = 0.0001
winReward = 1

In [300]:
class Omok:
    # --------------------------------
    # 초기화
    # --------------------------------
    def __init__(self, gridSize):
        self.gridSize = gridSize
        self.nbStates = self.gridSize * self.gridSize
        self.state = np.zeros(self.nbStates, dtype=np.uint8)
        self.curPlayer = 1
        
    # --------------------------------
    # 리셋
    # --------------------------------
    def reset(self):
        self.state = np.zeros(self.nbStates, dtype=np.uint8)
        
    # --------------------------------
    # 현재 상태 구함
    # --------------------------------
    def getState(self):
        return np.reshape(self.state, (1, self.nbStates))
    
    # --------------------------------
    # render
    # --------------------------------
    def render(self):
        state = np.reshape(self.state,(self.gridSize,self.gridSize))
        print(state)
        return state
    
    # --------------------------------
    # 매칭 검사
    # 다섯개 붙어 있으면 true, 아님 false
    # --------------------------------
    def CheckMatch(self, player):
        for y in range(self.gridSize):
            for x in range(self.gridSize):

                # --------------------------------
                # 오른쪽 검사
                # --------------------------------
                match = 0

                for i in range(STONE_MAX):
                    if (x + i >= self.gridSize):
                        break

                    if (self.state[y * self.gridSize + x + i] == player):
                        match += 1
                    else:
                        break;

                    if (match >= STONE_MAX):
                        return True

                # --------------------------------
                # 아래쪽 검사
                # --------------------------------
                match = 0

                for i in range(STONE_MAX):
                    if (y + i >= self.gridSize):
                        break

                    if (self.state[(y + i) * self.gridSize + x] == player):
                        match += 1
                    else:
                        break;

                    if (match >= STONE_MAX):
                        return True

                # --------------------------------
                # 오른쪽 대각선 검사
                # --------------------------------
                match = 0

                for i in range(STONE_MAX):
                    if ((x + i >= self.gridSize) or (y + i >= self.gridSize)):
                        break

                    if (self.state[(y + i) * self.gridSize + x + i] == player):
                        match += 1
                    else:
                        break;

                    if (match >= STONE_MAX):
                        return True

                # --------------------------------
                # 왼쪽 대각선 검사
                # --------------------------------
                match = 0

                for i in range(STONE_MAX):
                    if ((x - i < 0) or (y + i >= self.gridSize)):
                        break

                    if (self.state[(y + i) * self.gridSize + x - i] == player):
                        match += 1
                    else:
                        break;

                    if (match >= STONE_MAX):
                        return True

        return False
    
    # --------------------------------
    # 게임오버 검사
    # 리워드를 리턴
    # 이긴 player : 1, 진 player  : -1, 무승부 : 0
    # --------------------------------
    def getReward(self, player):
        if (self.CheckMatch(STONE_PLAYER1) == True):
            if (player == STONE_PLAYER1):
                return winReward
            else:
                return -winReward
        elif (self.CheckMatch(STONE_PLAYER2) == True):
            if (player == STONE_PLAYER1):
                return -winReward
            else:
                return winReward
        else:
            for i in range(self.nbStates):
                if (self.state[i] == STONE_NONE):
                    return 0
            return 0

        
    # --------------------------------
    # action취해서 state update
    # player : 다음 player로 전환
    # 이미 놓은 자리에 또 놓으면 음수 리워드
    # --------------------------------
    def getNextState(self, state, player, action):
        if(state[action] == 0):
            reward = 0
        else:
            reward = hadicapReward
        state[action] = player;
        self.state = state
        player = -player
        
        #return self.state, player
        return player, reward
    
    
    # --------------------------------
    # curPlayer에 맞는 state
    # --------------------------------
    def getCanonicalForm(self, state, curPlayer):
        return state * curPlayer
    
    # --------------------------------
    # 한 에피소드 동안 실행(게임이 끝날 때까지)
    # --------------------------------
    def step(self, sess):
        trainExamples = []
        self.curPlayer = 1
        
        episodeStep = 0 # array가 너무 깊어지면 안돌아가서.. 100번안에 episode가 안끝나면 그냥 그대로
        
        while True:
            print("step")
            episodeStep+=1
            
            self.render()
            
            canonicalBoard = self.getCanonicalForm(self.state,self.curPlayer)
            canonicalBoard = np.reshape(canonicalBoard,[1,nbStates])
            pi = sess.run(output_layer, feed_dict={X: canonicalBoard, keep_prob: 1.0})
            pi = np.reshape(pi,[nbStates,])
            
            trainExamples.append([self.state, self.curPlayer, pi])
            action = np.random.choice(len(pi), p=pi)
            
            #self.state, self.curPlayer = self.getNextState(self.state, self.curPlayer, action)
            self.curPlayer, reward = self.getNextState(self.state, self.curPlayer, action)
            
            if(reward == 0):
                r = self.getReward(self.curPlayer)
            else:
                r = reward
            
            if r!=0 or episodeStep==50:
                return [(x[0],x[2],r*((-1)**(x[1]!=self.curPlayer))) for x in trainExamples]
                #return trainExamples
            
                # 전체 에피소드 (한 수 한 수....)
                # 각 수마다 state, policy, reward
                # 마지막에 이긴 player가 둔 수에는 모두 r=1, 진 player가 둔 수는 모두 r=-1

In [301]:
# network architecture

X = tf.placeholder(tf.float32, [None, nbStates]) # state
x_image = tf.reshape(X, [-1, gridSize, gridSize, 1])

Y = tf.placeholder('float', [None, nbStates]) # action
adv = tf.placeholder('float') # reward

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

W_conv1 = weight_variable([3, 3, 1, 32])
b_conv1 = bias_variable([32])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_variable([3, 3, 32, 64])
b_conv2 = bias_variable([64])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

W_fc1 = weight_variable([2 * 2 * 64, 1024])
b_fc1 = bias_variable([1024])

h_pool2_flat = tf.reshape(h_pool2, [-1, 2*2*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder("float")
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_variable([1024, nbActions])
b_fc2 = bias_variable([nbActions])

output_layer=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

log_p = Y * tf.log(output_layer)
log_lik = log_p * adv
loss = tf.reduce_mean(tf.reduce_sum(-log_lik, axis=1))
train = tf.train.AdamOptimizer(LEARNING_RATE).minimize(loss)

In [302]:
def discount_rewards(r):
    '''Discounted reward를 구하기 위한 함수
    
    Args:
         r(np.array): reward 값이 저장된 array
    
    Returns:
        discounted_r(np.array): Discounted 된 reward가 저장된 array
    '''
    discounted_r = np.zeros_like(r, dtype=np.float32)
    running_add = 0
    for t in reversed(range(len(r))):
        running_add = running_add * DISCOUNT + r[t]
        discounted_r[t] = running_add

    return discounted_r

In [303]:
with tf.Session() as sess:
    game = Omok(gridSize)
    sess.run(tf.global_variables_initializer())
    cost = []
    
    for i in range(100):
        print("===============================")
        print("===============================")
        
        trainExamples = game.step(sess)
        trainExamples = np.array(trainExamples)
        
        rewards = discount_rewards(np.vstack(trainExamples[:,2]))
        
        cost_, _ = sess.run([loss,train], feed_dict={X: np.vstack(trainExamples[:,0]), Y: np.vstack(trainExamples[:,1]), adv : rewards})
        cost.append(cost_)

step
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]]
step
[[0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0]]
step
[[  0   0 255   0   0   0   0]
 [  0   0   0   0   0   1   0]
 [  0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0]]
step
[[  0   0 255   0   0   0   0]
 [  0   0   0   0   0   1   0]
 [  0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0]
 [  0   0   1   0   0   0   0]
 [  0   0   0   0   0   0   0]
 [  0   0   0   0   0   0   0]]


InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_29' with dtype float
	 [[Node: Placeholder_29 = Placeholder[dtype=DT_FLOAT, shape=<unknown>, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'Placeholder_29', defined at:
  File "/Users/apple/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/apple/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/apple/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/apple/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/Users/apple/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/Users/apple/anaconda3/lib/python3.6/site-packages/zmq/eventloop/ioloop.py", line 